# Лабораторная работа по теме Итерационные методы решения слау

### Вариант 4
Метод наискорейшего градиентного спуска

##### описание метода
Метод наискорейшего градиентного спуска применяется для решения системы линейных алгебраических уравнений (СЛАУ) вида: $Ax=b$ \
где \
A - симметричная положительно определенная матрица n×n \
x — вектор неизвестных \
b — вектор правых частей

In [285]:
import numpy as np
from typing import Tuple

In [286]:
def solve(
    A: np.ndarray,
    b: np.ndarray,
    tol: float = 1e-6,
    max_iter: int = 1000,
    with_logging: bool = False,
) -> Tuple[np.ndarray]:
    """
    Решение системы методом наискорейшего градиентного спуска
    residual - невязка
    """
    
    """
    этап 1:
    готовим начальные x и невязку
    для отслеживания насколько близко мы находимся к решению
    """
    n = A.shape[0]
    x = np.zeros(n)
    residual = A @ x - b
    
    
    for k in range(max_iter):
        """
        этап 2:
        вычисляем AResidual - насколько матрица A "усиливает" вектор невязки для расчёта оптимального шага alpha
        Вычисление оптимальный шаг alpha
        и обновляем решение
        """
        AResidual = A @ residual
        alpha = np.dot(residual, residual) / np.dot(residual, AResidual)
        x_new = x - alpha * residual
        
        """
        этап 3:
        вычисляем новую невязку по новому решению
        проверяем достигли ли мы нужной точности решения
        если да, то заканчиваем и выводим
        если нет, то обновляем переменные для следующих итераций
        """
        residual_new = A @ x_new - b
        if np.linalg.norm(residual_new) < tol:
            x = x_new
            residual = residual_new
            break

        x = x_new
        residual = residual_new
        
        # Вывод информации
        if with_logging and (k+1) % 100 == 0:
            print(f"Итерация {k+1}: норма невязки = {np.linalg.norm(residual_new):.6e}")
    
    else:
        if with_logging:
            print(f"Достигнуто максимальное число итераций {max_iter}")

    if with_logging:
        print(f"success={np.linalg.norm(residual) < tol}, iteratins={k+1}, residual={np.linalg.norm(residual):.6e}")
    
    return x
    

In [287]:
def generate(size: int = 4, tryings: int = 10**5) -> tuple[np.ndarray, np.ndarray]:
    for _ in range(tryings):
        # A_random = np.random.randn(size, size)
        A_random = np.random.randint(-10, 10, size=(size, size))
        A = (A_random + A_random.T)

        eigvals = np.linalg.eigvals(A)
        if np.any(eigvals <= 0):
            continue
        
        # b = np.random.randn(size)
        b = np.random.randint(-10, 10, size=(size))
        if np.all(np.linalg.solve(A, b)):
            return A, b

In [ ]:
A, b = generate(size=3)
A, b

(array([[12,  1,  0],
        [ 1, 12,  5],
        [ 0,  5, 18]]),
 array([  2,  -6, -10]))

In [289]:
my_solving = solve(
    A=A,
    b=b,
    tol=1e-10,
    max_iter=10**3,
    with_logging=True
)
my_solving

success=True, iteratins=25, residual=5.273527e-11


array([ 0.19349164, -0.32189974, -0.46613896])

In [290]:
np_solving = np.linalg.solve(A, b)
np_solving

array([ 0.19349164, -0.32189974, -0.46613896])

In [291]:
max_error = 0
for np_x, my_x in zip(np_solving, my_solving):
    max_error = max(max_error, abs(np_x - my_x))
max_error

np.float64(3.5058622671613193e-12)